In [11]:
from functools import wraps
import errno
import os
import signal

class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

In [12]:
import decorator
import time

def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get('timeout', 0.0) # seconds
    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for numTry in xrange(howmany):
            try: return func(*fargs, **fkwargs)
            except exception_types or Exception:
                #print('Retrying - ' + str(numTry+1))
                if timeout is not None: time.sleep(timeout)
                if numTry == howmany-1:
                    #print('Failed - Contuniuing...')
                    return 
    return tryIt

In [13]:
import numpy as np
import librosa
import random as rnd

rnd.seed(666)

@timeout(1)
def compute_melspectrogram(y, sr):
    return(librosa.feature.melspectrogram(y, sr=sr, n_fft=1024, hop_length=512, n_mels=128))

@retry(1, TimeoutError, timeout=0.0)
def compute_mel_and_retry_on_timeout(y, sr):
    return(compute_melspectrogram(y, sr))

In [14]:
import numpy as np
import librosa
import random as rnd

rnd.seed(666)

def computeAudioFeatures(audio_path):
    try:
        full_duration = librosa.get_duration(filename=audio_path)
    except EOFError or Exception:
        #print("Can't get duration!")
        return(np.zeros((128,599)))
    cut_duration = 13.9
    if full_duration < cut_duration:
        #print("Clip too short!")
        return(np.zeros((128,599)))

    max_cut_point = full_duration - cut_duration

    cut_offset=rnd.uniform(0.0, max_cut_point)

    # Load sound file
    y, sr = librosa.load(audio_path,duration=13.9,offset=cut_offset)
    # compute mel spectro
    S = compute_mel_and_retry_on_timeout(y, sr)
    if S is None:
        return(np.zeros((128,599)))
    else:
        # Convert to log scale (dB). We'll use the peak power as reference.
        log_S = librosa.logamplitude(S, ref_power=np.max)
        return(log_S)


In [27]:
from tqdm import tqdm
import pandas as pd
import pickle
import random as rnd

rnd.seed(666)

audio_dir = "/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_clips/"
file_list = librosa.util.find_files(audio_dir)
#82695
#file_list = file_list[0:20000]
#file_list = file_list[20001:40000]
#file_list = file_list[40001:60000]
#file_list = file_list[60001:82694]
files_per_round=2000
iters = int(np.floor(len(file_list)/files_per_round))
cur_indic = 0
for i in xrange(iters+1):
    if i==iters:
        cur_list = file_list[(i*files_per_round):(len(file_list)-1)]
    else:
        cur_list = file_list[(i*files_per_round):(((i+1)*files_per_round)-1)]
    clip_list = [(audio_path.split('audio_clips/')[1].split('.')[0], pd.Series(computeAudioFeatures(audio_path).flatten())) for audio_path in tqdm(cur_list)]    
    audio_features_df = pd.DataFrame.from_items(clip_list)
    audio_features_df = audio_features_df.loc[:, (audio_features_df != 0).any(axis=0)]
    with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/processedAudioFeatures/audio_features_part'+str(i)+'.pkl', mode='wb') as f:
        pickle.dump(audio_features_df, f)
    cur_indic+=2000
    

#clip_list = [(audio_path.split('audio_clips/')[1].split('.')[0], pd.Series(computeAudioFeatures(audio_path).flatten())) for audio_path in tqdm(file_list)]

#print('Done1 !')

#audio_features_df = pd.DataFrame.from_items(clip_list)

#print('Done1.5 !')

#with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_features_temp_part1.pkl', mode='wb') as f:
#with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_features_temp_part2.pkl', mode='wb') as f:
#with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_features_temp_part3.pkl', mode='wb') as f:
#with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_features_temp_part4.pkl', mode='wb') as f:
#    pickle.dump(audio_features_df, f)


print('Done1.75 !')

# Clean failed features
#audio_features_df = audio_features_df.loc[:, (audio_features_df != 0).any(axis=0)]

print('Done2 !')

#with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_features_part1.pkl', mode='wb') as f:
#with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_features_part2.pkl', mode='wb') as f:
#with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_features_part3.pkl', mode='wb') as f:
#with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_features_part4.pkl', mode='wb') as f:
#    pickle.dump(audio_features_df, f)

    
print('Done3 !')
    



  0%|          | 0/1999 [00:00<?, ?it/s]

  0%|          | 1/1999 [00:00<20:21,  1.64it/s]

  0%|          | 2/1999 [00:01<20:03,  1.66it/s]

  0%|          | 3/1999 [00:01<19:44,  1.69it/s]

  0%|          | 5/1999 [00:02<19:33,  1.70it/s]


  0%|          | 7/1999 [00:03<17:02,  1.95it/s]

  0%|          | 8/1999 [00:04<17:21,  1.91it/s]

  0%|          | 9/1999 [00:05<22:50,  1.45it/s]

  1%|          | 10/1999 [00:05<21:35,  1.54it/s]

  1%|          | 12/1999 [00:06<18:24,  1.80it/s]

  1%|          | 13/1999 [00:07<18:37,  1.78it/s]

  1%|          | 15/1999 [00:08<18:35,  1.78it/s]

  1%|          | 16/1999 [00:08<18:26,  1.79it/s]

  1%|          | 17/1999 [00:09<18:38,  1.77it/s]

  1%|          | 18/1999 [00:09<18:21,  1.80it/s]

  1%|          | 19/1999 [00:09<13:56,  2.37it/s]

  1%|          | 20/1999 [00:10<15:08,  2.18it/s]

  1%|          | 22/1999 [00:11<13:35,  2.43it/s]

  1%|          | 23/1999 [00:11<10:56,  3.01it/s]

  1%|▏         | 25/1999 [00:11<10:25,  3.16

 10%|▉         | 196/1999 [01:11<12:23,  2.43it/s]

 10%|▉         | 197/1999 [01:12<13:29,  2.23it/s]

 10%|▉         | 199/1999 [01:12<10:20,  2.90it/s]

 10%|█         | 201/1999 [01:13<09:49,  3.05it/s]

 10%|█         | 202/1999 [01:13<07:48,  3.84it/s]

 10%|█         | 203/1999 [01:13<10:22,  2.89it/s]

 10%|█         | 204/1999 [01:14<12:02,  2.49it/s]

 10%|█         | 205/1999 [01:14<13:22,  2.24it/s]

 10%|█         | 207/1999 [01:15<11:57,  2.50it/s]

 10%|█         | 208/1999 [01:15<13:20,  2.24it/s]

 11%|█         | 210/1999 [01:16<12:02,  2.48it/s]

 11%|█         | 211/1999 [01:17<13:13,  2.25it/s]

 11%|█         | 213/1999 [01:17<12:17,  2.42it/s]

 11%|█         | 214/1999 [01:18<13:47,  2.16it/s]

 11%|█         | 215/1999 [01:18<14:10,  2.10it/s]

 11%|█         | 217/1999 [01:19<12:20,  2.41it/s]

 11%|█         | 219/1999 [01:19<11:03,  2.68it/s]

 11%|█         | 221/1999 [01:20<08:14,  3.60it/s]

 11%|█         | 223/1999 [01:20<06:19,  4.68it/s]

 11%|█      

 20%|██        | 400/1999 [02:16<08:40,  3.07it/s]

 20%|██        | 401/1999 [02:16<10:11,  2.62it/s]

 20%|██        | 402/1999 [02:17<11:02,  2.41it/s]

 20%|██        | 404/1999 [02:17<08:22,  3.17it/s]

 20%|██        | 405/1999 [02:17<09:35,  2.77it/s]

 20%|██        | 407/1999 [02:17<07:15,  3.65it/s]

 20%|██        | 409/1999 [02:18<05:46,  4.58it/s]

 21%|██        | 411/1999 [02:18<06:07,  4.32it/s]

 21%|██        | 413/1999 [02:18<04:56,  5.35it/s]

 21%|██        | 415/1999 [02:19<05:52,  4.49it/s]

 21%|██        | 416/1999 [02:19<07:53,  3.34it/s]

 21%|██        | 417/1999 [02:21<14:34,  1.81it/s]

 21%|██        | 418/1999 [02:21<13:59,  1.88it/s]

 21%|██        | 419/1999 [02:22<13:59,  1.88it/s]

 21%|██        | 420/1999 [02:22<13:38,  1.93it/s]

 21%|██        | 421/1999 [02:23<13:11,  1.99it/s]

 21%|██        | 422/1999 [02:23<13:18,  1.98it/s]

 21%|██        | 423/1999 [02:24<13:12,  1.99it/s]

 21%|██        | 424/1999 [02:24<13:41,  1.92it/s]

 21%|██▏    

 30%|██▉       | 598/1999 [03:25<15:23,  1.52it/s]

 30%|███       | 600/1999 [03:25<11:11,  2.08it/s]

 30%|███       | 601/1999 [03:26<11:19,  2.06it/s]

 30%|███       | 602/1999 [03:26<09:04,  2.56it/s]

 30%|███       | 603/1999 [03:26<09:55,  2.34it/s]

 30%|███       | 605/1999 [03:28<10:57,  2.12it/s]

 30%|███       | 606/1999 [03:28<11:16,  2.06it/s]

 30%|███       | 607/1999 [03:29<11:15,  2.06it/s]

 30%|███       | 608/1999 [03:29<11:10,  2.07it/s]

 30%|███       | 609/1999 [03:30<11:31,  2.01it/s]

 31%|███       | 611/1999 [03:30<10:03,  2.30it/s]

 31%|███       | 613/1999 [03:31<08:49,  2.62it/s]

 31%|███       | 614/1999 [03:31<09:22,  2.46it/s]

 31%|███       | 615/1999 [03:32<09:57,  2.32it/s]

 31%|███       | 617/1999 [03:32<07:38,  3.02it/s]

 31%|███       | 618/1999 [03:32<08:38,  2.66it/s]

 31%|███       | 619/1999 [03:33<13:27,  1.71it/s]

 31%|███       | 621/1999 [03:34<10:05,  2.27it/s]

 31%|███       | 623/1999 [03:35<11:14,  2.04it/s]

 31%|███▏   

 40%|███▉      | 797/1999 [04:27<08:47,  2.28it/s]

 40%|███▉      | 798/1999 [04:28<09:01,  2.22it/s]

 40%|████      | 800/1999 [04:28<06:48,  2.94it/s]

 40%|████      | 801/1999 [04:28<07:36,  2.62it/s]

 40%|████      | 803/1999 [04:29<05:45,  3.46it/s]

 40%|████      | 804/1999 [04:29<04:39,  4.27it/s]

 40%|████      | 805/1999 [04:29<06:01,  3.30it/s]

 40%|████      | 806/1999 [04:30<07:14,  2.75it/s]

 40%|████      | 808/1999 [04:30<05:32,  3.58it/s]

 41%|████      | 810/1999 [04:30<05:28,  3.62it/s]

 41%|████      | 812/1999 [04:31<04:27,  4.43it/s]

 41%|████      | 813/1999 [04:31<06:00,  3.29it/s]

 41%|████      | 814/1999 [04:32<07:05,  2.78it/s]

 41%|████      | 816/1999 [04:32<06:36,  2.98it/s]

 41%|████      | 817/1999 [04:33<07:37,  2.59it/s]

 41%|████      | 819/1999 [04:33<06:55,  2.84it/s]

 41%|████      | 820/1999 [04:33<06:26,  3.05it/s]

 41%|████      | 822/1999 [04:34<06:07,  3.20it/s]

 41%|████      | 823/1999 [04:34<07:04,  2.77it/s]

 41%|████▏  

...

 68%|██████▊   | 1360/1999 [07:41<01:58,  5.40it/s]

 68%|██████▊   | 1362/1999 [07:41<01:38,  6.47it/s]

 68%|██████▊   | 1363/1999 [07:41<01:28,  7.16it/s]

 68%|██████▊   | 1365/1999 [07:42<01:53,  5.59it/s]

 68%|██████▊   | 1367/1999 [07:42<02:11,  4.80it/s]

 68%|██████▊   | 1368/1999 [07:43<03:07,  3.36it/s]

 68%|██████▊   | 1369/1999 [07:43<02:34,  4.09it/s]

 69%|██████▊   | 1370/1999 [07:43<03:20,  3.14it/s]

 69%|██████▊   | 1371/1999 [07:43<02:39,  3.93it/s]

 69%|██████▊   | 1372/1999 [07:43<02:14,  4.68it/s]

 69%|██████▊   | 1374/1999 [07:45<03:27,  3.01it/s]

 69%|██████▉   | 1375/1999 [07:45<04:02,  2.57it/s]

 69%|██████▉   | 1377/1999 [07:45<03:04,  3.38it/s]

 69%|██████▉   | 1378/1999 [07:46<02:32,  4.07it/s]

 69%|██████▉   | 1380/1999 [07:46<02:01,  5.08it/s]

 69%|██████▉   | 1382/1999 [07:46<01:40,  6.16it/s]

 69%|██████▉   | 1384/1999 [07:46<01:23,  7.35it/s]

 69%|██████▉   | 1386/1999 [07:48<03:22,  3.03it/s]

 69%|██████▉   | 1388/1999 [07:48<02:32,  4.01

 77%|███████▋  | 1549/1999 [08:43<03:43,  2.01it/s]

 78%|███████▊  | 1550/1999 [08:43<02:52,  2.60it/s]

 78%|███████▊  | 1551/1999 [08:43<02:16,  3.29it/s]

 78%|███████▊  | 1552/1999 [08:43<02:50,  2.63it/s]

 78%|███████▊  | 1553/1999 [08:44<03:10,  2.34it/s]

 78%|███████▊  | 1554/1999 [08:44<02:55,  2.53it/s]

 78%|███████▊  | 1555/1999 [08:45<04:24,  1.68it/s]

 78%|███████▊  | 1556/1999 [08:46<04:22,  1.68it/s]

 78%|███████▊  | 1557/1999 [08:46<04:16,  1.72it/s]

 78%|███████▊  | 1558/1999 [08:47<04:11,  1.75it/s]

 78%|███████▊  | 1559/1999 [08:47<04:05,  1.79it/s]

 78%|███████▊  | 1561/1999 [08:48<03:31,  2.07it/s]

 78%|███████▊  | 1562/1999 [08:49<03:35,  2.03it/s]

 78%|███████▊  | 1563/1999 [08:49<03:42,  1.96it/s]

 78%|███████▊  | 1564/1999 [08:49<02:48,  2.58it/s]

 78%|███████▊  | 1565/1999 [08:50<03:07,  2.32it/s]

 78%|███████▊  | 1566/1999 [08:50<02:26,  2.96it/s]

 78%|███████▊  | 1567/1999 [08:50<02:44,  2.62it/s]

 78%|███████▊  | 1568/1999 [08:50<02:09,  3.33

 86%|████████▋ | 1728/1999 [09:46<01:56,  2.32it/s]

 86%|████████▋ | 1729/1999 [09:46<02:05,  2.16it/s]

 87%|████████▋ | 1730/1999 [09:47<02:11,  2.05it/s]

 87%|████████▋ | 1731/1999 [09:47<02:13,  2.01it/s]

 87%|████████▋ | 1733/1999 [09:47<01:40,  2.64it/s]

 87%|████████▋ | 1734/1999 [09:48<01:19,  3.34it/s]

 87%|████████▋ | 1736/1999 [09:48<01:21,  3.24it/s]

 87%|████████▋ | 1737/1999 [09:49<02:25,  1.80it/s]

 87%|████████▋ | 1738/1999 [09:50<02:23,  1.82it/s]

 87%|████████▋ | 1740/1999 [09:50<01:45,  2.46it/s]

 87%|████████▋ | 1741/1999 [09:50<01:36,  2.67it/s]

 87%|████████▋ | 1742/1999 [09:51<01:46,  2.42it/s]

 87%|████████▋ | 1744/1999 [09:51<01:39,  2.57it/s]

 87%|████████▋ | 1746/1999 [09:52<01:14,  3.40it/s]

 87%|████████▋ | 1747/1999 [09:52<01:29,  2.82it/s]

 87%|████████▋ | 1748/1999 [09:53<01:40,  2.49it/s]

 87%|████████▋ | 1749/1999 [09:53<01:48,  2.30it/s]

 88%|████████▊ | 1750/1999 [09:53<01:24,  2.96it/s]

 88%|████████▊ | 1751/1999 [09:54<01:37,  2.55

 96%|█████████▌| 1919/1999 [10:53<00:36,  2.21it/s]

 96%|█████████▌| 1921/1999 [10:53<00:27,  2.83it/s]

 96%|█████████▌| 1922/1999 [10:54<00:43,  1.76it/s]

 96%|█████████▌| 1923/1999 [10:55<00:42,  1.78it/s]

 96%|█████████▌| 1924/1999 [10:55<00:32,  2.30it/s]

 96%|█████████▋| 1926/1999 [10:55<00:29,  2.51it/s]

 96%|█████████▋| 1928/1999 [10:56<00:21,  3.30it/s]

 96%|█████████▋| 1929/1999 [10:56<00:25,  2.70it/s]

 97%|█████████▋| 1931/1999 [10:57<00:23,  2.87it/s]

 97%|█████████▋| 1933/1999 [10:57<00:22,  2.99it/s]

 97%|█████████▋| 1934/1999 [10:58<00:25,  2.58it/s]

 97%|█████████▋| 1935/1999 [10:58<00:27,  2.32it/s]

 97%|█████████▋| 1937/1999 [10:58<00:19,  3.10it/s]

 97%|█████████▋| 1938/1999 [10:59<00:24,  2.52it/s]

 97%|█████████▋| 1940/1999 [10:59<00:17,  3.28it/s]

 97%|█████████▋| 1941/1999 [11:00<00:21,  2.68it/s]

 97%|█████████▋| 1943/1999 [11:00<00:16,  3.48it/s]

 97%|█████████▋| 1944/1999 [11:00<00:19,  2.80it/s]

 97%|█████████▋| 1945/1999 [11:01<00:21,  2.46

Done1.75 !
Done2 !
Done3 !


In [26]:
#np.squeeze(np.asarray(computeAudioFeatures(audio_path))).shape
#(np.squeeze(computeAudioFeatures(audio_path).reshape(1,-1))).shape
#(computeAudioFeatures(audio_path).reshape(1,-1)).shape
clip_number


NameError: name 'clip_number' is not defined

In [2]:
# Original shape (128,599)
# Array shape (1, 76672)

#    
#audio_features_df = audio_features_df.loc[:, (audio_features_df != 0).any(axis=0)]
#audio_features_df.head()
print 1

1


In [58]:
#import pandas as pd
#import pickle
#import random as rnd
#
#rnd.seed(666)
#
#audio_dir = "/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_clips/"
#file_list = librosa.util.find_files(audio_dir)
#
#audio_features_array_size = 76672
#last_column=audio_features_array_size+1
#clip_number = len(file_list)
#audio_features_df = pd.DataFrame(np.zeros((clip_number, audio_features_array_size+1)))
#audio_features_df.columns = [['clip_id'] + [(i+1) for i in range(audio_features_array_size)]]
#    
#i = 0
#for audio_path in file_list:
#    print('Working on row : ', i+1)
#    #clip_id = audio_path.split('audio_clips/')[1].split('.')[0]
#    #audio_features_df.iloc[i,0] = audio_path.split('audio_clips/')[1].split('.')[0]
#    #features = computeAudioFeatures(audio_path).flatten()
#    #audio_features_df.iloc[i,1:] = computeAudioFeatures(audio_path).flatten()
#    #audio_features_df = audio_features_df.append(([clip_id] + features.tolist()) , ignore_index=True)
#    i += 1
#    #82696
#    if i==3:
#        break
#
#print('Done!')
# 
##print audio_features_df
##with open('/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_features.pkl', mode='wb') as f:
##    pickle.dump(audio_features_df, f)
#
#

In [16]:
np.floor(len(file_list)/2000)

41.0

In [15]:
audio_dir = "/Users/dmarcous/git/wavenetmusicrecommendation/data/audio_clips/"
file_list = librosa.util.find_files(audio_dir)

In [20]:
for i in xrange(41): print i

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
